# Tensorflow Math Modules

tf.InteractiveSession을 사용해서 직접 값을 확인하도록 하겠다.   
InteractiveSession을 사용한 후에는 close를 통해서 닫아줘야 한다.   
필요에 따라서 사용하기 바란다.  
x.eval()은 sess.run(x)와 동일하다.  

In [6]:
import tensorflow as tf
    
# 실행의 간편함과 코드의 간결함을 위해서 InteractiveSession을 사용하도록 하겠다.
interactiveSession = tf.InteractiveSession()

x = tf.constant([3,6])

print(x.eval())

# is eqaul to

with tf.Session() as sess:
    print(sess.run(x))

[3 6]
[3 6]


In [5]:
interactiveSession.close()

## DType 주의 사항

* 텐서플로우의 데이터 타입은 Numpy와 동일하다.
* Python 타입이 타입 추론에 의해서 텐서플로우 타입으로 변경되지만, Python 타입은 사용하지 않는 것이 좋다.


## 1. Math

### 1.1 Baisc Operations

* tf.add
* tf.add_n
* tf.multiply
* tf.matmul
* tf.div
* tf.mod


In [7]:
x = tf.constant([3,6])
y = tf.constant([2,2])

print (tf.add(x, y).eval())
print (tf.add_n([x, y, y]).eval())
print (tf.multiply(x, y).eval())

print (tf.matmul(tf.reshape(x, shape=(1,2)), tf.reshape(y, shape=(1,2)), transpose_a=True).eval())

print (tf.div(x, y).eval())
print (tf.mod(x, y).eval())

[5 8]
[ 7 10]
[ 6 12]
[[ 6  6]
 [12 12]]
[1 3]
[1 0]


### 1.2 Reduction

리스트를 받아서 axis를 기준으로 평균을 반환

```python
tf.reduce_mean(
    input_tensor,
    axis=None,
    keepdims=None,
    name=None,
    reduction_indices=None,
    keep_dims=None
)
```

In [19]:
x = tf.constant([[1., 1.], [2., 2.]])

print(tf.reduce_mean(x).eval())
print(tf.reduce_mean(x, axis=0).eval())
print(tf.reduce_mean(x, axis=1).eval())


1.5
[1.5 1.5]
[1. 2.]


### 1.3 Sequence Comparison and Indexing
해당 range에서 가장 큰 index를 반환 한다.

```python
tf.argmax(
    input,
    axis=None,
    name=None,
    dimension=None,
    output_type=tf.int64
)
```

In [22]:
y = tf.constant([[0,0,1,0], [0,0,0,1], [0,0,0,1]]) # one hot encoding
yhat = tf.constant([[0.3,0.0,0.7,0.5], [0.4,0.2,0.1,0.6], [0.0,0.2,0.1,0.4]]) # predictions

y_index = tf.argmax(y, axis=1)
yhat_index = tf.argmax(yhat, axis=1)

print('y: ', y_index.eval())
print('yhat: ', yhat_index.eval())

correct_preds = tf.equal(y_index, yhat_index)

print('correct_preds: ', correct_preds.eval())

accuracy = tf.reduce_sum(tf.cast(correct_preds, dtype=tf.float32))

print('accuracy: ', accuracy.eval())



y:  [2 3 3]
yhat:  [2 3 3]
correct_preds:  [ True  True  True]
accuracy:  3.0


### 1.4 Random

tensorflow는 여러 분산으로 부터 랜덤 tensor를 생성하는 라이브러리를 제공한다.  
자주 사용하는 함수는 다음과 같으니 익혀두는 것이 좋다.

* tf.random_normal
* tf.truncated_noraml
* tf.random_uniform
* tf.random_shuffle
* tf.random_crop
* tf.multinomial
* tf.random_gamma

랜덤 값을 재 생성하기 위해서는 seed값을 지정해야 한다.
다음 매소드를 사용해 그래프 레벨의 seed를 지정할 수 있다.

tf.set_random_seed(seed)

조금 상세한 설명이 필요하지만 아주 간단히 설명하면 그래프 seed는 전체 seed로 사용되고 개발 매소드 seed는 개별로 사용된다.  
랜덤 seed관련해서는 [링크](https://www.tensorflow.org/api_docs/python/tf/set_random_seed) 참고하기 바란다.


대표적으로 많이 사용되는 truncated_normal를 살펴보겠다.
truncated_normal은 random_normal과 동일하게 정규 분포를 따르는 랜덤 값을 반환 한다.   
두드러지는 특징은 딥러닝 초기화에 사용할 경우, 표준 편차 2가 넘는 값을 버리기 때문에 truncated_normal가 더 좋은 성능을 보인다.

```python
tf.truncated_normal(
    shape,
    mean=0.0,
    stddev=1.0,
    dtype=tf.float32,
    seed=None,
    name=None
)
```

In [23]:
tf.set_random_seed(0)
norm = tf.truncated_normal(shape=(2, 3), mean=1, stddev=1)
norm.eval()

# 그래프 seed를 0으로 지정했기 때문에 항상 첫번째 값은 항상 다음과 같다.
# [[-0.40955448,  0.46331722,  0.43476212],[ 1.5262461 ,  0.82560146,  1.2242969 ]]

array([[ 2.489018  , -0.09126794,  0.84315634],
       [ 0.24372363,  1.4334596 ,  0.27243507]], dtype=float32)

다음은 시퀀스의 인덱스를 랜덤하게 변경한다.  
이미지의 시퀀스를 변경할 때 유용하게 사용할 수 있다.

```python
tf.random_shuffle(
    value,
    seed=None,
    name=None
)
```

In [ ]:
c = tf.constant([[1, 2], [3, 4], [5, 6]])
shuff = tf.random_shuffle(c)
shuff.eval()

## 2. Loss Function

### 2.1. Cross entropy

* Binary classification
\\[ -{(y\log(p) + (1 - y)\log(1 - p))} \\]

```python
tf.losses.sigmoid_cross_entropy(
    multi_class_labels,
    logits,
    weights=1.0,
    label_smoothing=0,
    scope=None,
    loss_collection=tf.GraphKeys.LOSSES,
    reduction=Reduction.SUM_BY_NONZERO_WEIGHTS
)
```

* Multiclass classification

\\[ -\sum_{c=1}^My_{o,c}\log(p_{o,c}) \\]

```python
tf.nn.softmax_cross_entropy_with_logits(
    _sentinel=None,
    labels=None,
    logits=None,
    dim=-1,
    name=None
)
```



In [26]:
# batch size을 3으로 가정한다.
# logits은 sigmoid의 입력 값이다. [0~1] 사이 값이 아니다.
logits = tf.constant([[-4,-4,-4,4,-4], [-4,-4,-4,-4,4], [-5,-2,-1,-1,5]], dtype=tf.float32)
labels = tf.constant([[0,0,0,1,0], [0,0,0,0,1], [0,0,0,0,1]])

# 5개에 대한 클래스에 모든 에러를 더한다. 
entropy = tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits, labels=labels, name='loss')
print(entropy.eval())

# entropy / class 갯수 = 평균 entropy (loss)
loss = tf.reduce_mean(entropy)
print(loss.eval())

[0.00134092 0.00134092 0.00589738]
0.0028597414


### 2.2. Noise-contrastive estimation 

상세한 내용은 [링크](http://proceedings.mlr.press/v9/gutmann10a/gutmann10a.pdf)를 참고하기 바란다.

```python
tf.nn.nce_loss(
    weights,
    biases,
    labels,
    inputs,
    num_sampled,
    num_classes,
    num_true=1,
    sampled_values=None,
    remove_accidental_hits=False,
    partition_strategy='mod',
    name='nce_loss'
)
```